In [47]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ast

pd.set_option("display.width", 300)
pd.set_option("display.max_columns", 14)

# 'Movie' Dataset

## Load 'Movie' Dataset

In [48]:
movie_data = pd.read_csv("Rotten Tomatoes reviews/rotten_tomatoes_movies.csv")
movie_data["releaseDateStreaming"] = pd.to_datetime(movie_data["releaseDateStreaming"], format="%Y-%m-%d", errors="coerce")
movie_data["releaseDateTheaters"] = pd.to_datetime(movie_data["releaseDateTheaters"], format="%Y-%m-%d", errors="coerce")
print(movie_data.head())
print(movie_data.shape[0])
print(movie_data.dtypes)

                     id                title  audienceScore  tomatoMeter rating ratingContents releaseDateTheaters  ...                          genre  originalLanguage                       director                                  writer boxOffice distributor soundMix
0    space-zombie-bingo  Space Zombie Bingo!           50.0          NaN    NaN            NaN                 NaT  ...         Comedy, Horror, Sci-fi           English                  George Ormrod              George Ormrod,John Sabotta       NaN         NaN      NaN
1       the_green_grass      The Green Grass            NaN          NaN    NaN            NaN                 NaT  ...                          Drama           English                Tiffany Edwards                         Tiffany Edwards       NaN         NaN      NaN
2             love_lies           Love, Lies           43.0          NaN    NaN            NaN                 NaT  ...                          Drama            Korean  Park Heung-Sik,He

## Clean 'Movie' Dataset

In [49]:
movie_data.drop_duplicates(subset=["id"], inplace=True)

### Drop Entries with no Movie Title

In [50]:
no_title = movie_data[pd.isnull(movie_data["title"])].shape[0]
movie_data.drop(movie_data[pd.isnull(movie_data["title"])].index, inplace=True)

print(f"Movies dropped due to not having a title in the data: {no_title}")

Movies dropped due to not having a title in the data: 365


In [51]:
print("Movies with missing title after cleaning:", movie_data[pd.isnull(movie_data["title"])].shape[0])

Movies with missing title after cleaning: 0


### Create "ReleaseYear" Column

In [52]:
movie_data["releaseYear"] = movie_data["releaseDateTheaters"].dt.year
movie_data["releaseYear"] = movie_data["releaseYear"].combine_first(movie_data["releaseDateStreaming"].dt.year)
print(movie_data["releaseYear"].describe())

count    83513.000000
mean      2007.672291
std         19.830149
min       1902.000000
25%       2006.000000
50%       2016.000000
75%       2018.000000
max       2032.000000
Name: releaseYear, dtype: float64


#### Check Movies with Future Release Dates

In [53]:
print(movie_data[movie_data["releaseYear"] > 2025])

# Correct Release Date
movie_data.loc[movie_data["id"] == "other_peoples_children_2022", ["releaseDateTheaters", "releaseYear"]] = [pd.to_datetime("21.09.2022", dayfirst=True), 2022]
print(movie_data.loc[movie_data["id"] == "other_peoples_children_2022", ["releaseDateTheaters", "releaseYear"]])

                                 id                    title  audienceScore  tomatoMeter rating ratingContents releaseDateTheaters  ... originalLanguage           director             writer boxOffice           distributor soundMix releaseYear
36508                      avatar_4                 Avatar 4            NaN          NaN    NaN            NaN          2026-12-18  ...          English      James Cameron      James Cameron       NaN  20th Century Studios      NaN      2026.0
50971                      avatar_5                 Avatar 5            NaN          NaN    NaN            NaN          2028-12-22  ...          English      James Cameron      James Cameron       NaN  20th Century Studios      NaN      2028.0
123693  other_peoples_children_2022  Other People's Children            NaN         92.0    NaN            NaN          2032-04-21  ...           French  Rebecca Zlotowski  Rebecca Zlotowski       NaN       Music Box Films      NaN      2032.0

[3 rows x 17 columns]
 

### Transform Genre Data Column into List containing all Genres the Movie fits into

In [54]:
movie_data["genre"] = movie_data["genre"].str.replace(pat=", ", repl= ",", regex=True).str.split(",")
print("Movies with no Genre Data available:", movie_data[~pd.isnull(movie_data["genre"])].shape[0])

Movies with no Genre Data available: 130986


### Transform ratingContents Column into List

In [55]:
movie_data["ratingContents"] = movie_data["ratingContents"].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [])
print("Movies with no Genre Data available:", movie_data[~pd.isnull(movie_data["ratingContents"])].shape[0])

Movies with no Genre Data available: 141687


### Check for presence of Box Office Data

In [56]:
print("Number of 'boxOffice' entries:", movie_data[~pd.isnull(movie_data["boxOffice"])].shape[0])

Number of 'boxOffice' entries: 14614


### Extract numerical Value from 'boxOffice'

In [57]:
movie_data[["boxOffice_numerical", "multiplier"]] = movie_data["boxOffice"].str.extract(r"(\d+.\d+)([MK])")
movie_data.loc[movie_data["multiplier"] == "M", "multiplier"] = 1000000
movie_data.loc[movie_data["multiplier"] == "K", "multiplier"] = 1000
movie_data["boxOffice_numerical"] = pd.to_numeric(movie_data["boxOffice_numerical"])
movie_data["multiplier"] = pd.to_numeric(movie_data["multiplier"])
movie_data["boxOffice_numerical"] = movie_data["boxOffice_numerical"] * movie_data["multiplier"]
movie_data.drop(columns=["multiplier"], axis=1, inplace=True)
print(movie_data.head(20))
print(movie_data.dtypes)

                                      id                                             title  audienceScore  tomatoMeter rating                                     ratingContents releaseDateTheaters  ...                       director                                            writer boxOffice  \
0                     space-zombie-bingo                               Space Zombie Bingo!           50.0          NaN    NaN                                                 []                 NaT  ...                  George Ormrod                        George Ormrod,John Sabotta       NaN   
1                        the_green_grass                                   The Green Grass            NaN          NaN    NaN                                                 []                 NaT  ...                Tiffany Edwards                                   Tiffany Edwards       NaN   
2                              love_lies                                        Love, Lies           43.0       

### Import Scraped 'Box Office' Dataset

In [58]:
Box_Office = pd.read_csv("Rotten Tomatoes reviews/RT BoxOffice/BoxOffice_numerical_full_Dataset_404s_missing.csv")
print(Box_Office.head())

   Unnamed: 0              id   BoxOffice
0           0       unbranded         NaN
1           1  1053155-lassie   8300000.0
2           2           klute         NaN
3           3        the_duff  34000000.0
4           4    falling_down  40400000.0


### Drop "Unnamed" Columns

In [59]:
Box_Office.drop(labels=["Unnamed: 0"], inplace=True, axis=1)
print(Box_Office.head())
print(Box_Office.dtypes)

               id   BoxOffice
0       unbranded         NaN
1  1053155-lassie   8300000.0
2           klute         NaN
3        the_duff  34000000.0
4    falling_down  40400000.0
id            object
BoxOffice    float64
dtype: object


### Add Box Office Data into Movie Dataset

In [60]:
movie_data = pd.merge(left = movie_data, right=Box_Office, on = "id", how="left")
print("Number of Box Office figures supplemented through scraping:", movie_data[(np.isnan(movie_data["boxOffice_numerical"])) & (~np.isnan(movie_data["BoxOffice"]))].shape[0])

Number of Box Office figures supplemented through scraping: 100


### Combine scraped and already existing Box Office Data into a single column

In [61]:
movie_data["BoxOffice"] = movie_data["BoxOffice"].combine_first(movie_data["boxOffice_numerical"])
movie_data.drop(columns=["boxOffice", "boxOffice_numerical"], axis=1, inplace=True)
movie_data.rename(columns={"BoxOffice": "boxOffice"}, inplace=True)
print("Number of Movies with Box Office data:", movie_data[~np.isnan(movie_data["boxOffice"])].shape[0])
print(movie_data.head())

Number of Movies with Box Office data: 14599
                     id                title  audienceScore  tomatoMeter rating ratingContents releaseDateTheaters  ... originalLanguage                       director                                  writer distributor soundMix releaseYear boxOffice
0    space-zombie-bingo  Space Zombie Bingo!           50.0          NaN    NaN             []                 NaT  ...          English                  George Ormrod              George Ormrod,John Sabotta         NaN      NaN      2018.0       NaN
1       the_green_grass      The Green Grass            NaN          NaN    NaN             []                 NaT  ...          English                Tiffany Edwards                         Tiffany Edwards         NaN      NaN      2020.0       NaN
2             love_lies           Love, Lies           43.0          NaN    NaN             []                 NaT  ...           Korean  Park Heung-Sik,Heung-Sik Park  Ha Young-Joon,Jeon Yun-su,Song Hy

### Remove irrelevant columns from 'Movie' Dataframe, rename column containing the age rating

In [62]:
movie_data.drop(labels=["soundMix", "distributor", "writer"], axis=1, inplace=True)
movie_data.rename(columns={"rating": "ageRating"}, inplace=True)
print(movie_data.columns)

Index(['id', 'title', 'audienceScore', 'tomatoMeter', 'ageRating', 'ratingContents', 'releaseDateTheaters', 'releaseDateStreaming', 'runtimeMinutes', 'genre', 'originalLanguage', 'director', 'releaseYear', 'boxOffice'], dtype='object')


### Check Descriptive Statistics for Missing Values

In [63]:
movie_data.describe(include='all')

,id,title,audienceScore,tomatoMeter,ageRating,ratingContents,releaseDateTheaters,releaseDateStreaming,runtimeMinutes,genre,originalLanguage,director,releaseYear,boxOffice
count,141687,141687,72628.000000,33574.000000,13855,141687,30501,78689,128275.000000,130986,128237,137795,83513.000000,1.459900e+04
unique,141687,126403,NaN,NaN,10,8353,NaN,NaN,NaN,2911,112,62186,NaN,NaN
top,space-zombie-bingo,The Return,NaN,NaN,R,[],NaN,NaN,NaN,[Drama],English,Unknown Director,NaN,NaN
freq,1,17,NaN,NaN,7665,127832,NaN,NaN,NaN,27605,84277,3515,NaN,NaN
mean,NaN,NaN,55.669356,65.764818,NaN,NaN,1995-06-07 01:28:11.472410880,2014-06-25 15:12:46.811117312,93.703902,NaN,NaN,NaN,2007.672171,1.838298e+07
min,NaN,NaN,0.000000,0.000000,NaN,NaN,1902-10-04 00:00:00,1928-12-18 00:00:00,1.000000,NaN,NaN,NaN,1902.000000,3.690000e+02
25%,NaN,NaN,37.000000,45.000000,NaN,NaN,1981-04-03 00:00:00,2010-02-23 00:00:00,84.000000,NaN,NaN,NaN,2006.000000,5.420000e+04
50%,NaN,NaN,57.000000,73.000000,NaN,NaN,2006-01-23 00:00:00,2016-11-05 00:00:00,92.000000,NaN,NaN,NaN,2016.000000,6.734000e+05
75%,NaN,NaN,76.000000,89.000000,NaN,NaN,2016-09-09 00:00:00,2018-12-06 00:00:00,103.000000,NaN,NaN,NaN,2018.000000,1.430000e+07
max,NaN,NaN,100.000000,100.000000,NaN,NaN,2028-12-22 00:00:00,2023-12-22 00:00:00,2700.000000,NaN,NaN,NaN,2028.000000,8.584000e+08


# 'Critic Reviews' Dataset

## Load 'Critic Reviews' Data

In [64]:
critic_reviews = pd.read_csv("Rotten Tomatoes reviews/rotten_tomatoes_critic_reviews.csv")
critic_reviews.rename(columns={"publicatioName": "publicationName", "originalScore": "originalRating"}, inplace=True)
critic_reviews["id"] = pd.Categorical(critic_reviews["id"])
critic_reviews["creationDate"] = pd.to_datetime(critic_reviews["creationDate"])
print(critic_reviews.head())
print("\n", critic_reviews.dtypes)
print("\nTotal Number of Observations", critic_reviews.shape[0])
print(f"\nTotal Number of Movies: {critic_reviews['id'].unique().shape[0]}")

                                  id  reviewId creationDate       criticName  isTopCritic originalRating reviewState                publicationName                                         reviewText scoreSentiment                                          reviewUrl
0                            beavers   1145982   2003-05-23  Ivan M. Lincoln        False          3.5/4       fresh  Deseret News (Salt Lake City)  Timed to be just long enough for most youngste...       POSITIVE  http://www.deseretnews.com/article/700003233/B...
1                         blood_mask   1636744   2007-06-02    The Foywonder        False            1/5      rotten                  Dread Central  It doesn't matter if a movie costs 300 million...       NEGATIVE  http://www.dreadcentral.com/index.php?name=Rev...
2  city_hunter_shinjuku_private_eyes   2590987   2019-05-28     Reuben Baron        False            NaN       fresh                            CBR  The choreography is so precise and lifelike at...       

### Load Scraped Critic Reviews

In [65]:
scraped_critic_reviews = pd.concat([pd.read_json(f"Rotten Tomatoes reviews/Critic Reviews Scraped/rt_critic_reviews_scraped_batch_{i}.json") for i in range(0, 200)], ignore_index=True)
scraped_critic_reviews.rename(columns={"quote":"reviewText", "originalScore": "originalRating"}, inplace=True)
scraped_critic_reviews.drop(columns=["publicationUrl", "isRtUrl", "isRotten", "isFresh", "criticPageUrl", "criticPictureUrl"], axis=1, inplace=True)
scraped_critic_reviews["creationDate"] = pd.to_datetime(scraped_critic_reviews["creationDate"])
scraped_critic_reviews["id"] = pd.Categorical(scraped_critic_reviews["id"])
print(scraped_critic_reviews.head())
print("Number of Scraped Reviews:", scraped_critic_reviews.shape[0])

  creationDate          criticName reviewState  isTopCritic       publicationName                                          reviewUrl                                         reviewText  reviewId originalRating scoreSentiment        id
0   2021-02-02      Richard Crouse       fresh        False        Richard Crouse  http://www.richardcrouse.ca/the-duff-3-stars-a...  A school comedy so predictable the screenwrite...   2763233            3/5       POSITIVE  the_duff
1   2020-05-27        Andrew Galdi       fresh        False         Movie Bitches  https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...               It was good acting and good writing.   2692661                      POSITIVE  the_duff
2   2020-03-26       Avaryl Halley       fresh        False         Movie Bitches  https://www.youtube.com/watch?v=8KeoWwUtXVQ&li...                            Oh dear, am I the Duff?   2679627                      POSITIVE  the_duff
3   2019-08-15  Udita Jhunjhunwala       fresh         True     

### Find Scraped Reviews not already Contained in the 'Critic Reviews' Dataset

In [66]:
df_all = scraped_critic_reviews.merge(critic_reviews, on="reviewId", how="left", indicator=True)
scraped_only = df_all[df_all["_merge"] == "left_only"].reset_index(drop=True)
print("Number of Scraped Reviews not found in the open source Dataset:", scraped_only.shape[0])

Number of Scraped Reviews not found in the open source Dataset: 44467


### Prepare 'scraped_only' Reviews to be inserted into 'Critic Reviews' Dataset

In [67]:
scraped_only.rename(columns={"id_x": "id", "reviewText_x": "reviewText", "creationDate_x": "creationDate", "criticName_x": "criticName", "reviewState_x": "reviewState", "isTopCritic_x": "isTopCritic", "publicationName_x": "publicationName", "reviewUrl_x": "reviewUrl", "originalRating_x": "originalRating", "scoreSentiment_x": "scoreSentiment"}, inplace=True)
scraped_only.drop(columns=["id_y", "reviewText_y", "creationDate_y", "criticName_y", "isTopCritic_y", "publicationName_y", "reviewUrl_y", "originalRating_y", "scoreSentiment_y", "reviewState_y", "_merge"], inplace=True)

print(scraped_only.head())

  creationDate     criticName reviewState  isTopCritic publicationName                                          reviewUrl                                         reviewText   reviewId originalRating scoreSentiment               id
0   2024-10-10  Bianca Garner       fresh        False       Filmotomy  https://filmotomy.com/rewind-1993-in-film-the-...  Its technical aspects along with Douglas’ perf...  102956494           None       POSITIVE     falling_down
1   2011-07-18           None       fresh        False        TV Guide    http://movies.tvguide.com/falling/review/129657  These adventures would be offensive if you cou...    1998091            3/4       POSITIVE     falling_down
2   2011-07-18           None       fresh        False           Film4     http://www.film4.com/reviews/1992/falling-down  What makes this an innovative film is Joel Sch...    1998089                      POSITIVE     falling_down
3   2006-01-26           None       fresh         True        Time Out  http

### Insert Scraped Critic Reviews into Dataset

In [68]:
critic_reviews = pd.concat([critic_reviews, scraped_only], ignore_index=True)
print("Total Number fo unique Critic Reviews:", critic_reviews.shape[0])
print("\n", critic_reviews.head())

Total Number fo unique Critic Reviews: 1489430

                                   id  reviewId creationDate       criticName  isTopCritic originalRating reviewState                publicationName                                         reviewText scoreSentiment                                          reviewUrl
0                            beavers   1145982   2003-05-23  Ivan M. Lincoln        False          3.5/4       fresh  Deseret News (Salt Lake City)  Timed to be just long enough for most youngste...       POSITIVE  http://www.deseretnews.com/article/700003233/B...
1                         blood_mask   1636744   2007-06-02    The Foywonder        False            1/5      rotten                  Dread Central  It doesn't matter if a movie costs 300 million...       NEGATIVE  http://www.dreadcentral.com/index.php?name=Rev...
2  city_hunter_shinjuku_private_eyes   2590987   2019-05-28     Reuben Baron        False            NaN       fresh                            CBR  The cho

## Clean 'Critic Reviews' Dataset

### Create Column Containing the Length of the Written Review

In [69]:
critic_reviews["reviewLength"] = critic_reviews["reviewText"].str.len()
critic_reviews["reviewLength"] = pd.to_numeric(critic_reviews["reviewLength"], errors="coerce")
print("Reviews with no Review Text:", critic_reviews[pd.isnull(critic_reviews["reviewLength"])].shape[0])

Reviews with no Review Text: 69227


### Remove Observations with no Review Text

In [70]:
reviews_before = critic_reviews.shape[0]
critic_reviews.drop(critic_reviews[pd.isnull(critic_reviews["reviewText"])].index, inplace=True)
critic_reviews.drop(critic_reviews[critic_reviews["reviewText"] == "."].index, inplace=True)
critic_reviews.drop(critic_reviews[critic_reviews["reviewLength"] <= 2].index, inplace=True)
reviews_after = critic_reviews.shape[0]

print(f"Critic Reviews lost due to no review text: {reviews_before-reviews_after}")

Critic Reviews lost due to no review text: 74750


### Drop reviews with a creation date before the launch of Rotten Tomatoes

In [71]:
critic_reviews.drop(critic_reviews[critic_reviews["creationDate"] < pd.to_datetime("1998-08-12", format="%Y-%m-%d")].index, inplace=True)
critic_reviews.reset_index(inplace=True, drop=True)

### Drop duplicate reviews

In [72]:
critic_reviews.drop_duplicates("reviewId", inplace=True)

### Extract Rating out of "originalScore" column, standardize Numerical Rating as a score out of ten

In [73]:
critic_reviews[["Rating", "RatingOutOf"]] = critic_reviews["originalRating"].str.split(pat = "/", n = 1, expand=True)
critic_reviews["Rating"] = pd.to_numeric(critic_reviews["Rating"], errors = "coerce")
critic_reviews["RatingOutOf"] = pd.to_numeric(critic_reviews["RatingOutOf"], errors = "coerce")
critic_reviews["ratingOutOfTen"] = 10 * critic_reviews["Rating"] / critic_reviews["RatingOutOf"]
critic_reviews.drop(labels=["Rating", "RatingOutOf"], axis=1, inplace=True)
print(critic_reviews.dtypes)

id                         object
reviewId                    int64
creationDate       datetime64[ns]
criticName                 object
isTopCritic                  bool
originalRating             object
reviewState                object
publicationName            object
reviewText                 object
scoreSentiment             object
reviewUrl                  object
reviewLength              float64
ratingOutOfTen            float64
dtype: object


### Remove negative scores and scores recorded outside the rating range

In [74]:
critic_reviews = critic_reviews.drop(critic_reviews[critic_reviews["ratingOutOfTen"] < 0].index)
critic_reviews = critic_reviews.drop(critic_reviews[critic_reviews["ratingOutOfTen"] > 10].index)

### Check Summary Statistics for Missing Values in important Columns

In [75]:
critic_reviews.describe(include="all")

,id,reviewId,creationDate,criticName,isTopCritic,originalRating,reviewState,publicationName,reviewText,scoreSentiment,reviewUrl,reviewLength,ratingOutOfTen
count,1402176,1.402176e+06,1402176,1396831,1402176,961704,1402176,1402176,1402176,1402176,1239003,1.402176e+06,794328.000000
unique,67607,NaN,NaN,16032,2,1707,2,2920,1397941,2,1151017,NaN,NaN
top,joker_2019,NaN,NaN,Dennis Schwartz,False,3/5,fresh,New York Times,Parental Content Review,POSITIVE,,NaN,NaN
freq,608,NaN,NaN,13092,971571,103402,942952,19587,239,942952,969,NaN,NaN
mean,NaN,1.080573e+07,2013-10-28 11:44:24.939066112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.321133e+02,6.421767
min,NaN,1.000000e+00,1998-08-14 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000e+00,0.000000
25%,NaN,1.710780e+06,2008-02-22 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.900000e+01,5.000000
50%,NaN,2.245798e+06,2015-02-06 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.310000e+02,6.250000
75%,NaN,2.620090e+06,2019-08-29 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.720000e+02,8.000000
max,NaN,1.030420e+08,2025-08-02 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.630000e+02,10.000000


## Merging the 'Critic Review' and 'Movie' Datasets

### Merge Critic Review Data with Movie Data

In [76]:
merged_data = pd.merge(left=critic_reviews, right=movie_data, how="left", on="id")
print(merged_data.head())

                                  id  reviewId creationDate       criticName  isTopCritic originalRating reviewState  ... releaseDateStreaming runtimeMinutes          genre originalLanguage          director  releaseYear boxOffice
0                            beavers   1145982   2003-05-23  Ivan M. Lincoln        False          3.5/4       fresh  ...           2011-06-21           30.0  [Documentary]          English       Stephen Low       2011.0       NaN
1                         blood_mask   1636744   2007-06-02    The Foywonder        False            1/5      rotten  ...                  NaT            NaN            NaN              NaN  Unknown Director          NaN       NaN
2  city_hunter_shinjuku_private_eyes   2590987   2019-05-28     Reuben Baron        False            NaN       fresh  ...                  NaT            NaN            NaN              NaN      Kenji Kodama          NaN       NaN
3  city_hunter_shinjuku_private_eyes   2558908   2019-02-14      Matt Schley

### Select Rows with Complete 'boxOffice' Information ('boxOffice' simultaneously drops movies with missing data in the 'movie' Dataset)

In [77]:
Working_Critic_Data = merged_data[~np.isnan(merged_data["boxOffice"])].copy()

print(f"Number of Observations lost due to no Box Office revenue: {merged_data.shape[0]-Working_Critic_Data.shape[0]}")
print(f"Number of Movies with no available Box Office revenue information: {merged_data['id'].drop_duplicates().count()-Working_Critic_Data['id'].drop_duplicates().count()}")

Number of Observations lost due to no Box Office revenue: 423936
Number of Movies with no available Box Office revenue information: 53721


### Select Observations for Movies with at least 20 critic reviews

In [78]:
total_observations = Working_Critic_Data.shape[0]
total_movies = Working_Critic_Data["id"].drop_duplicates().count()

Working_Critic_Data = Working_Critic_Data[Working_Critic_Data.groupby("id")["reviewText"].transform("count") >= 20]
Working_Critic_Data.reset_index(drop=True, inplace=True)
print("Observations lost due to fewer than 20 Critic reviews:", total_observations-Working_Critic_Data.shape[0])
print("Number of Movies with fewer than 20 Critic Reviews:", total_movies-Working_Critic_Data["id"].drop_duplicates().count())
print(f"Number of Observations left: {Working_Critic_Data.shape[0]}")
print(f"Total Movies for Webscraping: {Working_Critic_Data['id'].drop_duplicates().count()}")

Observations lost due to fewer than 20 Critic reviews: 40403
Number of Movies with fewer than 20 Critic Reviews: 4748
Number of Observations left: 937837
Total Movies for Webscraping: 9138


### Save 'Critic Reviews' Dataset for Webscraping of Audience Reviews

In [79]:
#Working_Critic_Data.to_csv("Rotten Tomatoes Reviews/critic_reviews_clean_for_emsId_20+_critic_reviews.csv")

# 'Audience Reviews' Dataset

## Load Scraped 'Audience Reviews' Dataset

In [80]:
audience_reviews = pd.concat([pd.read_json(f"Rotten Tomatoes reviews/Audience Reviews Scraped/rt_audience_reviews_scraped_batch_{i}.json") for i in range(0, 256)], ignore_index=True)
audience_reviews["creationDate"] = pd.to_datetime(audience_reviews["creationDate"])

print(audience_reviews.head())
audience_reviews.dtypes

   rating                                              quote                              reviewId  isVerified  isSuperReviewer  hasSpoilers  hasProfanity  ...  creationDate userRealm                                userId        id                                    userAccountLink  \
0     3.5  We think we live in a place without labels but...  701d7950-87f6-4f90-92da-2f407fb989d7       False            False        False         False  ...    2025-02-20  Flixster  6f9d08f2-2bc7-40fa-8856-56837b52e391  the_duff                                               None   
1     5.0              Hilarious movie with a great message!  1f00763c-a236-4192-8717-626c83b61889       False            False        False         False  ...    2025-02-11        RT  a4c896d2-1628-43e7-908b-1fff954a5c2a  the_duff  /profiles/6y8IW2igQFgmuB1F6XhL8cG4S99Cv6sjZhkK...   
2     4.0  It was really enjoyable, the chemistry was bel...  60289d85-23b1-4ee2-909a-3f20eeef2023       False            False        False  

rating                    float64
quote                      object
reviewId                   object
isVerified                   bool
isSuperReviewer              bool
hasSpoilers                  bool
hasProfanity                 bool
score                     float64
creationDate       datetime64[ns]
userRealm                  object
userId                     object
id                         object
userAccountLink            object
userDisplayName            object
userImageUrl               object
dtype: object

## Clean 'Audience Reviews' Dataset

### Drop Column 'userImageUrl', 'userAccountLink', 'userRealm' and Rename Columns to Match the 'Critic Reviews' Dataset

In [81]:
audience_reviews.drop(columns=["userImageUrl", "userAccountLink", "userRealm"], axis=1, inplace=True)
audience_reviews.rename(columns={"score": "ratingOutOfTen", "rating": "originalRating", "quote": "reviewText"}, inplace=True)
audience_reviews["ratingOutOfTen"] = audience_reviews["originalRating"] * 2
print(audience_reviews.head())
audience_reviews.dtypes

   originalRating                                         reviewText                              reviewId  isVerified  isSuperReviewer  hasSpoilers  hasProfanity  ratingOutOfTen creationDate                                userId        id userDisplayName
0             3.5  We think we live in a place without labels but...  701d7950-87f6-4f90-92da-2f407fb989d7       False            False        False         False             7.0   2025-02-20  6f9d08f2-2bc7-40fa-8856-56837b52e391  the_duff            None
1             5.0              Hilarious movie with a great message!  1f00763c-a236-4192-8717-626c83b61889       False            False        False         False            10.0   2025-02-11  a4c896d2-1628-43e7-908b-1fff954a5c2a  the_duff            Ed R
2             4.0  It was really enjoyable, the chemistry was bel...  60289d85-23b1-4ee2-909a-3f20eeef2023       False            False        False         False             8.0   2024-06-12  2f1bbe30-f2eb-41b5-9159-48ab346313b2  t

originalRating            float64
reviewText                 object
reviewId                   object
isVerified                   bool
isSuperReviewer              bool
hasSpoilers                  bool
hasProfanity                 bool
ratingOutOfTen            float64
creationDate       datetime64[ns]
userId                     object
id                         object
userDisplayName            object
dtype: object

### Add a Column containing the Length of the Review

In [82]:
audience_reviews["reviewLength"] = audience_reviews["reviewText"].str.len()

### Check for Reviews with no Review Text

In [83]:
print("Number of Audience Reviews with missing review text:", audience_reviews[pd.isnull(audience_reviews["reviewText"])].shape[0])

Number of Audience Reviews with missing review text: 0


### Check Minimum Length of Audience Reviews

In [84]:
print("Shortest Review Text Length:", audience_reviews["reviewLength"].min())

Shortest Review Text Length: 7


### Check Descriptive Statistics for Missing Values in Important Columns

In [85]:
print("Number of Observations:", audience_reviews.shape[0])
print("Number of Text Reviews:", audience_reviews.dropna(subset=["reviewText"]).shape[0])
audience_reviews.describe(include="all")

Number of Observations: 1954144
Number of Text Reviews: 1954144


,originalRating,reviewText,reviewId,isVerified,isSuperReviewer,hasSpoilers,hasProfanity,ratingOutOfTen,creationDate,userId,id,userDisplayName,reviewLength
count,1.954144e+06,1954144,1719340,1954144,1954144,1954144,1954144,1.954144e+06,1954144,1954144,1954144,379349,1.954144e+06
unique,NaN,1915530,1717621,2,2,2,2,NaN,NaN,574896,11134,68368,NaN
top,NaN,It's good movie to watch,2b0022ec-2327-4326-9a4d-3708f7de90e7,False,False,False,False,NaN,NaN,20f97638-9460-4b49-a86c-4f730c64ca60,the_duff,steve d,NaN
freq,NaN,660,2,1928938,1917897,1952484,1925595,NaN,NaN,3523,250,3055,NaN
mean,3.459241e+00,NaN,NaN,NaN,NaN,NaN,NaN,6.918482e+00,2015-10-28 01:09:32.405718528,NaN,NaN,NaN,2.974304e+02
min,5.000000e-01,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+00,1996-02-08 00:00:00,NaN,NaN,NaN,7.000000e+00
25%,2.500000e+00,NaN,NaN,NaN,NaN,NaN,NaN,5.000000e+00,2011-08-16 00:00:00,NaN,NaN,NaN,6.300000e+01
50%,3.500000e+00,NaN,NaN,NaN,NaN,NaN,NaN,7.000000e+00,2014-11-29 00:00:00,NaN,NaN,NaN,1.340000e+02
75%,4.500000e+00,NaN,NaN,NaN,NaN,NaN,NaN,9.000000e+00,2020-07-12 00:00:00,NaN,NaN,NaN,2.900000e+02
max,5.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+01,2025-08-04 00:00:00,NaN,NaN,NaN,6.553500e+04


### Generate 'reviewIds' for Reviews with missing Review Ids

In [86]:
i = 1
new_ids = []

for rid in audience_reviews["reviewId"]:
    if pd.isnull(rid) or rid == "":
        new_ids.append(f"custom_{i}")
        i += 1
    else:
        new_ids.append(rid)

audience_reviews["reviewId"] = new_ids

audience_reviews.drop_duplicates("reviewId", inplace=True)  # Ensure every review is only present once

## Merging the 'Audience Reviews' and 'Movie' Dataset

In [87]:
merged_audience_data = audience_reviews.merge(right=movie_data, how="left", on="id")
print(merged_audience_data.head())
merged_audience_data.dtypes

   originalRating                                         reviewText                              reviewId  isVerified  isSuperReviewer  hasSpoilers  hasProfanity  ...  releaseDateStreaming runtimeMinutes              genre originalLanguage    director  releaseYear   boxOffice
0             3.5  We think we live in a place without labels but...  701d7950-87f6-4f90-92da-2f407fb989d7       False            False        False         False  ...            2015-06-09           96.0  [Romance, Comedy]          English  Ari Sandel       2015.0  34000000.0
1             5.0              Hilarious movie with a great message!  1f00763c-a236-4192-8717-626c83b61889       False            False        False         False  ...            2015-06-09           96.0  [Romance, Comedy]          English  Ari Sandel       2015.0  34000000.0
2             4.0  It was really enjoyable, the chemistry was bel...  60289d85-23b1-4ee2-909a-3f20eeef2023       False            False        False         False  ..

originalRating                 float64
reviewText                      object
reviewId                        object
isVerified                        bool
isSuperReviewer                   bool
hasSpoilers                       bool
hasProfanity                      bool
ratingOutOfTen                 float64
creationDate            datetime64[ns]
userId                          object
id                              object
userDisplayName                 object
reviewLength                     int64
title                           object
audienceScore                  float64
tomatoMeter                    float64
ageRating                       object
ratingContents                  object
releaseDateTheaters     datetime64[ns]
releaseDateStreaming    datetime64[ns]
runtimeMinutes                 float64
genre                           object
originalLanguage                object
director                        object
releaseYear                    float64
boxOffice                

### Select Rows with Complete 'boxOffice' Information (Necessary because the list of movies for scraping was originally created before fully cleaning the 'critic reviews Dataset)

In [88]:
Working_Audience_Data = merged_audience_data[~np.isnan(merged_audience_data["boxOffice"])].copy()
print("Number of Observations lost due to unavailability of Box Office data:", merged_audience_data.shape[0]-Working_Audience_Data.shape[0])
print("Number of Movies with no available Box Office data:", merged_audience_data["id"].unique().shape[0]-Working_Audience_Data["id"].unique().shape[0])

Number of Observations lost due to unavailability of Box Office data: 284784
Number of Movies with no available Box Office data: 2014


# Compare / Match 'Critic Reviews' and 'Audience Reviews' Datasets

## Movies only found in the 'Audience Reviews' Dataset

In [89]:
audience_reviews_movies = Working_Audience_Data.drop_duplicates(subset=["id"], ignore_index=True)
critic_reviews_movies = Working_Critic_Data.drop_duplicates(subset=["id"], ignore_index=True)
df_all = audience_reviews_movies.merge(critic_reviews_movies, on="id", how="left", indicator=True)
audience_reviews_only = df_all[df_all["_merge"] == "left_only"]["id"].reset_index()
print("Number of Movies only found in 'Audience Reviews':", audience_reviews_only.shape[0])

Number of Movies only found in 'Audience Reviews': 0


## Movies only found in the 'Critic Reviews' Dataset

In [90]:
df_all = Working_Critic_Data.drop_duplicates(subset=["id"]).merge(audience_reviews, on="id", how="left", indicator=True)
critic_reviews_only = df_all[df_all["_merge"] == "left_only"]["id"].reset_index(drop=True)
print("Number of Movies only found in 'Critic Reviews' (404 Error or no audience reviews on rottentomatoes.com):", critic_reviews_only.shape[0])
print("\n", critic_reviews_only)

Number of Movies only found in 'Critic Reviews' (404 Error or no audience reviews on rottentomatoes.com): 22

 0     can_you_bring_it_bill_t_jones_and_d_man_in_the...
1                                   brimstone_and_glory
2                       trailer-park-boys-the-big-dirty
3                                          tremors_2019
4                                                   xxx
5                                          dolores_2017
6                             spark_a_burning_man_story
7                              this_ones_for_the_ladies
8                                     peter_rabbit_2018
9                                                scales
10                                     pearl_jam_twenty
11                                   the_candidate_2018
12                                        viktoria_2014
13                                         the_riot_act
14                                    the_painting_2013
15                                         creep_

## Drop Critic Reviews concerning Movies not found in the 'Audience Reviews' Dataset

In [91]:
Working_Critic_Data = Working_Critic_Data[~Working_Critic_Data["id"].isin(critic_reviews_only)]

#Check again for Movies only found in 'Critics Reviews'
df_all = Working_Critic_Data.drop_duplicates(subset=["id"]).merge(audience_reviews_movies, on="id", how="left", indicator=True)
critic_reviews_only = df_all[df_all["_merge"] == "left_only"]["id"].reset_index(drop=True)
print("Number of Movies only found in 'Critic Reviews' after cleaning:", critic_reviews_only.shape[0])

Number of Movies only found in 'Critic Reviews' after cleaning: 0


# Save Cleaned Datasets

## Save 'Movie' Dataset

### Select Movies present in the 'Critic Reviews' and 'Audience Reviews' Dataset

In [92]:
movies_in_review_data = Working_Critic_Data["id"].drop_duplicates()

print(f"Number of unique movies with reviews: {len(movies_in_review_data)}")

movie_data_clean = movie_data[movie_data["id"].isin(movies_in_review_data)]

print(f"Number of Movies before selection: {len(movie_data)}")
print(f"Number of Movies after selection: {len(movie_data_clean)}")

Number of unique movies with reviews: 9116
Number of Movies before selection: 141687
Number of Movies after selection: 9116


### Save 'Movies' Dataset

In [93]:
movie_data_clean.to_json("Rotten Tomatoes Reviews/rt_movies_clean.json", date_format="iso", orient="records", indent=2)

## Save Cleaned 'Audience Reviews' Dataset for Future Translation of Non-English Reviews

In [94]:
print(f"Number of Audience Reviews: {Working_Audience_Data.shape[0]}")
n_batches = 20

# Group by id, so all reviews concerning the same movie are in the same batch
groups = [g for _, g in Working_Audience_Data[["reviewId", "id", "title", "reviewText", "originalRating", "ratingOutOfTen", "creationDate", "userId"]].groupby("id")]

# Initialize List for allocation and list of batch size to get an even split
chunks = [[] for _ in range(n_batches)]
sizes = [0] * n_batches

# Always assign the reviews concerning to the movie to the currently smallest batch
for g in groups:
    idx = np.argmin(sizes)
    chunks[idx].append(g)
    sizes[idx] += len(g)

# Save each batch as JSON
for i, chunk in enumerate(chunks):
    batch = pd.concat(chunk, ignore_index=True)
    batch.to_json(f"Rotten Tomatoes reviews/Audience Reviews pre Translation/rt_audience_reviews_pre_translation_{i}.json", date_format="iso", orient="records", indent=2)
    print(f"Batch {i} saved as json")

Number of Audience Reviews: 1667641
Batch 0 saved as json
Batch 1 saved as json
Batch 2 saved as json
Batch 3 saved as json
Batch 4 saved as json
Batch 5 saved as json
Batch 6 saved as json
Batch 7 saved as json
Batch 8 saved as json
Batch 9 saved as json
Batch 10 saved as json
Batch 11 saved as json
Batch 12 saved as json
Batch 13 saved as json
Batch 14 saved as json
Batch 15 saved as json
Batch 16 saved as json
Batch 17 saved as json
Batch 18 saved as json
Batch 19 saved as json


## Save Cleaned 'Critic Reviews' Dataset for Future Translation of Non-English Reviews

In [95]:
print(f"Number of Critic Reviews: {Working_Critic_Data.shape[0]}")
n_batches = 10

# Group by id, so all reviews concerning the same movie are in the same batch
groups = [g for _, g in Working_Critic_Data[["reviewId", "id", "title", "reviewText", "originalRating", "reviewState", "ratingOutOfTen", "creationDate", "criticName"]].groupby("id")]

# Initialize List for allocation and list of batch size to get an even split
chunks = [[] for _ in range(n_batches)]
sizes = [0] * n_batches

# Always assign the reviews concerning to the movie to the currently smallest batch
for g in groups:
    idx = np.argmin(sizes)
    chunks[idx].append(g)
    sizes[idx] += len(g)

# Save each batch as JSON
for i, chunk in enumerate(chunks):
    batch = pd.concat(chunk, ignore_index=True)
    batch.to_json(f"Rotten Tomatoes reviews/Critic Reviews pre Translation/rt_critic_reviews_pre_translation_{i}.json", date_format="iso", orient="records", indent=2)
    print(f"Batch {i} saved as json")

Number of Critic Reviews: 937037
Batch 0 saved as json
Batch 1 saved as json
Batch 2 saved as json
Batch 3 saved as json
Batch 4 saved as json
Batch 5 saved as json
Batch 6 saved as json
Batch 7 saved as json
Batch 8 saved as json
Batch 9 saved as json
